In [15]:
import pandas as pd
import numpy as np
import requests
import shutil
import gzip
import io

#### Realizando download do Arquivo

In [16]:
url  = 'https://data.brasil.io/dataset/covid19/caso_full.csv.gz'
local= 'C:/Users/fabio.pedrosa/Documents/pessoal/Laboratorio_02/analise_covid_19/fabio'
s = requests.Session()
r = s.get(url)
with gzip.open(io.BytesIO(r.content), 'r') as f_in, open('caso_full.csv', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

#### Lendo o dataset 

In [4]:
df = pd.read_csv('caso_full.csv',sep=',',parse_dates=['date']).sort_values('city_ibge_code')
df.dropna(inplace=True)
df.head(5)

NameError: name 'pd' is not defined

In [5]:
df_municipio = pd.read_csv('municipio.csv',sep=',').sort_values('codigo_uf')
df_municipio.head(5)

NameError: name 'pd' is not defined

In [6]:
df_estado = pd.read_csv('estado.csv',sep=',').sort_values('codigo_uf')
df_estado.rename(columns={"latitude": "latitude_uf", "longitude": "longitude_uf",'nome':'nome_uf'},inplace=True)
df_estado.head(5)

NameError: name 'pd' is not defined

#### Juntando os dataframe de estado e município

In [7]:
temp=df_municipio.merge(df_estado, left_on='codigo_uf', right_on='codigo_uf',how='inner')
colunas = ['capital','codigo_ufcodigo_uf']
temp=temp.sort_values('codigo_ibge')
temp.head(5)

NameError: name 'df_municipio' is not defined

#### Jutando os dados do covid-19 com o data frame de município

In [8]:
temp=df.merge(temp, left_on='city_ibge_code', right_on='codigo_ibge',how='inner')
temp.head(5)

NameError: name 'df' is not defined

In [9]:
colunas = ['city', 'city_ibge_code', 'epidemiological_week',
       'estimated_population_2019', 'is_last', 'is_repeated',
       'last_available_confirmed_per_100k_inhabitants', 'last_available_date',
       'last_available_death_rate',  'order_for_place',
       'place_type', 'state', 'codigo_ibge']
temp.drop(columns=colunas,axis=1,inplace=True)
temp.describe()

NameError: name 'temp' is not defined

In [10]:
temp['ativos']= (temp.last_available_confirmed - temp.last_available_deaths) + temp.new_confirmed
temp.head(5)

NameError: name 'temp' is not defined

#### Obtem o numero de casos confirmados, mortes, recuperados e ativos agrupado por data

In [11]:
import warnings
warnings.filterwarnings('ignore')

temp_agg = temp.groupby(['date'])['last_available_confirmed','last_available_deaths','new_confirmed', 'new_deaths', 'ativos'].sum().reset_index()
temp_agg.head(5)

NameError: name 'temp' is not defined

#### Remodela o dataframe com variável e valor para ter quantidades de recuperados, mortos e ativos

In [12]:
temp_01 = temp_agg.melt(id_vars="date", value_vars=['new_confirmed', 'new_deaths', 'ativos'],
                 var_name='casos', value_name='Quantidade')
temp_01.head(5)

NameError: name 'temp_agg' is not defined

In [13]:
# habilita modo offline
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.express as px
init_notebook_mode(connected=True)

In [13]:
# Definindo o renderizador para abrir numa nova aba:
#import plotly.io as pio
#pio.renderers
#pio.renderers.default = "browser"

In [13]:
# Cores
new_confirmed = '#21bf73'
new_deaths = '#ff2e63'
ativos = '#fe9801'

#### Plotando o gráfico de quantida de casos ao longo do tempo

In [1]:
fig = px.area(temp_01, 
              x="date", 
              y="Quantidade", 
              color='casos', 
              height=600,
              title='Casos ao longo do tempo',
              color_discrete_sequence = [new_confirmed, new_deaths, ativos])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

NameError: name 'px' is not defined

#### Agrupando os dados de covid-19 por data e cidade

In [15]:
temp_agg = temp.groupby(['date', 'uf','codigo_uf','latitude_uf','longitude_uf'])['new_confirmed', 'new_deaths', 'ativos'].sum().reset_index()
temp_agg.head(5)

,date,uf,codigo_uf,latitude_uf,longitude_uf,new_confirmed,new_deaths,ativos
0,2020-02-25,SP,35,-22.19,-48.79,1,0,2
1,2020-02-26,SP,35,-22.19,-48.79,0,0,1
2,2020-02-27,SP,35,-22.19,-48.79,0,0,1
3,2020-02-28,SP,35,-22.19,-48.79,1,0,3
4,2020-02-29,SP,35,-22.19,-48.79,0,0,2


In [14]:
import numpy as np
temp_agg.rename(columns={'latitude_uf':'latitude','longitude_uf':'longitude'},inplace=True)
temp_agg.sort_values('ativos', ascending=False)

NameError: name 'temp_agg' is not defined

#### Plotando o gráfico de casos no decorrer do tempo

In [2]:
fig = px.density_mapbox(temp_agg,
                        title='Casos ao longo do tempo',
                        lat='latitude', 
                        lon='longitude', 
                        z=np.log(temp_agg["ativos"]),
                        #radius=10,
                        hover_name='uf',
                        hover_data=["new_confirmed", "new_deaths",'ativos'],
                        center={"lat": -15.83, "lon": -47.86}, 
                        zoom=3,
                        color_continuous_scale="Viridis",
                        animation_frame=temp_agg["date"].dt.strftime('%Y-%m-%d'),
                        mapbox_style="open-street-map"
                       )
fig.update_layout(autosize=False, width=1200, height=800)   
fig.show()

NameError: name 'px' is not defined

In [20]:
import json

with open('br_states.json') as f:
    estados = json.load(f)
#counties

In [3]:
import plotly.graph_objects as go
fig = go.Figure(go.Choroplethmapbox(geojson=estados,
                                    locations=temp_agg.uf,
                                    z=temp_agg.ativos,
                                    colorscale="Viridis", 
                                    marker_opacity=0.5, 
                                    marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3, mapbox_center = {"lat": -15.83, "lon": -47.86})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

NameError: name 'estados' is not defined